In [ ]:
!pip install kaggle

## Upload kaggle.json file

In [ ]:
!mkdir - p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## Import Twitter sentiment dataset

In [ ]:
# API to fetch dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

 80% 65.0M/80.9M [00:00<00:00, 98.8MB/s]
100% 80.9M/80.9M [00:00<00:00, 109MB/s] 


In [ ]:
# extract the compressed dataset
from zipfile import ZipFile
dataset = "/content/sentiment140.zip"

with ZipFile(dataset, "r") as zip:
    zip.extractall()
    print("The dataset is extracted")

The dataset is extracted


## Import the dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# print the stopwords in english
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data Processing

In [ ]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding= "ISO-8859-1")

In [ ]:
# checking the number of row and columns
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns
column_names = ["target", "id", "date", "flag", "user", "text"]
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", names=column_names
                           ,encoding= "ISO-8859-1")

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# counting number of missing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
# check distribution of taget column
twitter_data["target"].value_counts()

0    800000
4    800000
Name: target, dtype: int64

## Convert the target 4 to 1

In [ ]:
twitter_data.replace({"target": {4: 1}}, inplace=True)

0 == Nagative tweet \
1 == Positive tweet

## Stemming
Stemming is a text preprocessing technique used in natural language processing (NLP) to reduce words to their root or base form.
#### examples: ate, eating, eaten = eat

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub("[^a-zA-Z],", " ", content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = " ".join(stemmed_content)

    return stemmed_content

In [ ]:
twitter_data["stemmed_content"] = twitter_data["text"].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset can't updat facebook text it... might cr...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan dive mani time ball. manag save 50% ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, behav all. i'm mad. here?..."


In [ ]:
# seperating the data and label
X = twitter_data["stemmed_content"].values
Y = twitter_data["target"].values

In [ ]:
print(X)

["@switchfoot http://twitpic.com/2y1zl - awww, that' bummer. shoulda got david carr third day it. ;d"
 "upset can't updat facebook text it... might cri result school today also. blah!"
 '@kenichan dive mani time ball. manag save 50% rest go bound' ...
 'readi mojo makeover? ask detail'
 'happi 38th birthday boo alll time!!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


#### splitting the data to traing set and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' "@hatermagazin i'm in!"
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 "hope everyon mother great day! can't wait hear guy store tomorrow!"
 'love wake folgers. bad voic deeper his.']


In [ ]:
print(X_test)

['@mmangen - fine! much time chat twitter. hubbi back summer &amp; tend domin free time.'
 'ah may show w/ ruth kim &amp; geoffrey sanhueza'
 '@ishatara mayb bay area thang dammit!' ...
 '@destini41 nevertheless hooray! 4700 member wonder safe trip!'
 'feel well' '@supersandro !!!! thank you!']


In [ ]:
# convert text to numerical data
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

  (0, 556009)	0.4487211454174097
  (0, 311986)	0.4213991907475672
  (0, 166835)	0.3784262004980888
  (0, 254280)	0.5248776010269405
  (0, 453142)	0.3579601505161932
  (0, 548776)	0.2724586226416628
  (1, 247293)	0.4217763793919321
  (1, 226276)	0.9066998873866882
  (2, 375395)	0.16689131329536258
  (2, 193782)	0.20184489556903598
  (2, 214803)	0.1869630826677776
  (2, 245860)	0.16108289611785123
  (2, 516809)	0.15260245515454093
  (2, 349447)	0.24085622070286994
  (2, 556418)	0.3361461884660533
  (2, 129250)	0.3117948063721909
  (2, 545166)	0.32796789247634894
  (2, 513686)	0.32017018023494215
  (2, 190534)	0.29198867181463134
  (2, 514669)	0.1860214627420369
  (2, 184436)	0.1903002181558718
  (2, 166835)	0.4613524893175993
  (3, 239589)	0.3833952637004258
  (3, 519500)	0.2688571630775124
  (3, 493051)	0.39144428130663844
  :	:
  (1279996, 410111)	0.21030006327121542
  (1279996, 315385)	0.22194158728144947
  (1279996, 378415)	0.17763524725553437
  (1279996, 520681)	0.1868101411948357
 

In [ ]:
print(X_test)

  (0, 530684)	0.17887835664019144
  (0, 516809)	0.315140720762036
  (0, 505288)	0.34595298252858964
  (0, 492720)	0.2184203857766327
  (0, 364094)	0.17667950995160334
  (0, 354538)	0.4494718454139425
  (0, 238384)	0.28692176527997193
  (0, 200586)	0.23596373707690663
  (0, 193842)	0.25329287074879764
  (0, 163026)	0.3737251779046731
  (0, 117471)	0.2669200768529063
  (0, 74681)	0.16105542998406155
  (0, 55607)	0.1704153426492517
  (1, 466660)	0.24404047137125223
  (1, 445368)	0.4817432021352679
  (1, 337854)	0.28401890370293914
  (1, 291642)	0.40705537655850976
  (1, 208906)	0.5712386084348501
  (1, 55607)	0.20816056254050785
  (1, 45590)	0.3039120772851599
  (2, 507254)	0.43372650860155376
  (2, 337944)	0.25870330976237066
  (2, 252268)	0.5874563217416909
  (2, 143686)	0.3624753396547525
  (2, 78460)	0.37869049381095454
  :	:
  (319995, 526826)	0.3993028113127016
  (319995, 417507)	0.3357991112090264
  (319995, 407189)	0.2582059877686021
  (319995, 384853)	0.40236168335324335
  (31999

## Logistic Regression

In [ ]:
model = LogisticRegression(max_iter = 1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

#### Model Evaluation

##### Accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print("Accuracy score on the training data : ", training_data_accuracy)

Accuracy score on the training data :  0.81680234375


In [ ]:
# accuracy score on the training data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print("Accuracy score on the test data : ", test_data_accuracy)

Accuracy score on the test data :  0.782303125


####*Accuracy score is 78.23%*